#Data Analytics Module 2 - Apache Spark Manipulations
##Loading CSV files into the databricks file system (dbfs)

In [0]:
%sh
mkdir -p /tmp/spark_lab

wget -O /tmp/spark_lab/2019.csv https://raw.githubusercontent.com/fanidam91/smartdata-lab/TEST/dev/Data_AI/Data_ai/2019_edited.csv
wget -O /tmp/spark_lab/2020.csv https://raw.githubusercontent.com/fanidam91/smartdata-lab/TEST/dev/Data_AI/Data_ai/2020_edited.csv
wget -O /tmp/spark_lab/2021.csv https://raw.githubusercontent.com/fanidam91/smartdata-lab/TEST/dev/Data_AI/Data_ai//2021_edited.csv

##Loading the CSV files into a dataframe

In [0]:
df = spark.read.load('spark_lab/*.csv', format='csv')
display(df.limit(100))

##Defining Schema for the dataframe

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])
df = spark.read.load('/spark_lab/*.csv', format='csv', schema=orderSchema)
display(df.limit(100))

##Cleaning the Data

In [0]:
from pyspark.sql.functions import col
df = df.dropDuplicates()
df = df.withColumn('Tax', col('UnitPrice') * 0.08)
df = df.withColumn('Tax', col('Tax').cast("float"))

##Creating a new Dataframe

In [0]:
customers_df = df.select("CustomerName", "Email", "Item", "Quantity")
customers_df = customers_df.withColumn("FirstName", split(customers_df["CustomerName"], " ").getItem(0))
customers_df = customers_df.withColumn("LastName", split(customers_df["CustomerName"], " ").getItem(1))

display(customers_df)

##Counting distinct customer entries

In [0]:

print(customers_df.count())
print(customers_df.distinct().count())

##Creating a Product Sales Dataframe

In [0]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()
display(productSales)
     

##Aggregating Yealy Sales

In [0]:

yearlySales = df.select(year("OrderDate").alias("Year")).groupBy("Year").count().orderBy("Year")
display(yearlySales)